Board Game Analysis for Adam Bartoszek
more explanation

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML

ab_collection_path = "./Collections/AB_Collection.csv"
jj_collection_path = "./Collections//JJ_Collection.csv"
bgg_collection_path = "./Collections//bgg_dataset.csv"
#mechanics_path = "./Database_Tables_BGG/mechanics.csv"
subcat_path = "./Database_Tables_BGG/subcategories.csv"
themes_path = "./Database_Tables_BGG/themes.csv"

jj_df = pd.read_csv(jj_collection_path, sep=',')
ab_df = pd.read_csv(ab_collection_path, sep=',')
bgg_df = pd.read_csv(bgg_collection_path, sep=';')
#mech_df = pd.read_csv(mechanics_path, sep=',')
sub_df = pd.read_csv(subcat_path, sep=',')
theme_df = pd.read_csv(themes_path, sep=',')

def filter_dataframe(df):
    # Filter out rows where 'itemtype' is not equal to 'expansion'
    filtered_df = df[df['itemtype'] != 'expansion']
    return filtered_df

def apply_suffix(df, suffix):
    # add suffix to all columns in DF to support easier separation and analysis
    return df.add_suffix(suffix)

jj_df_filtered = filter_dataframe(jj_df)
ab_df_filtered = filter_dataframe(ab_df)

jj_df_suffixed = apply_suffix(jj_df_filtered, '_jj')
ab_df_suffixed = apply_suffix(ab_df_filtered, '_ab')

ab_jj_outer = pd.merge(jj_df_suffixed, ab_df_suffixed, left_on='objectid_jj', right_on='objectid_ab', how='outer')
ab_jj_outer['objectid_comb'] = ab_jj_outer['objectid_ab'].combine_first(ab_jj_outer['objectid_jj'])
result = pd.merge(bgg_df, ab_jj_outer, left_on='ID', right_on='objectid_comb', how='left')

#print(jj_df_suffixed.columns)
#print(ab_df_suffixed.columns)
#print(ab_jj_outer.columns)
print(list(result.columns))
row_count = result.count().max()
print(f"Number of rows: {row_count}")

['ID', 'Name', 'Year Published', 'Min Players', 'Max Players', 'Play Time', 'Min Age', 'Users Rated', 'Rating Average', 'BGG Rank', 'Complexity Average', 'Owned Users', 'Mechanics', 'Domains', 'objectname_jj', 'objectid_jj', 'rating_jj', 'numplays_jj', 'weight_jj', 'own_jj', 'fortrade_jj', 'want_jj', 'wanttobuy_jj', 'wanttoplay_jj', 'prevowned_jj', 'preordered_jj', 'wishlist_jj', 'wishlistpriority_jj', 'wishlistcomment_jj', 'comment_jj', 'conditiontext_jj', 'haspartslist_jj', 'wantpartslist_jj', 'collid_jj', 'baverage_jj', 'average_jj', 'avgweight_jj', 'rank_jj', 'numowned_jj', 'objecttype_jj', 'originalname_jj', 'minplayers_jj', 'maxplayers_jj', 'playingtime_jj', 'maxplaytime_jj', 'minplaytime_jj', 'yearpublished_jj', 'bggrecplayers_jj', 'bggbestplayers_jj', 'bggrecagerange_jj', 'bgglanguagedependence_jj', 'publisherid_jj', 'imageid_jj', 'year_jj', 'language_jj', 'other_jj', 'itemtype_jj', 'barcode_jj', 'version_publishers_jj', 'version_languages_jj', 'version_yearpublished_jj', 'vers

In [ ]:
#Analysis
# Step 1: Count distinct values in 'ID' column
distinct_id_count = result['ID'].nunique()

# Step 2: Count distinct, non-NaN values in 'objectid_ab' column
dist_objectid_ab = result[result['itemtype_ab'] != 'expansion']['objectid_ab'].dropna().loc[result['objectid_ab'] != ''].nunique()
dist_objectid_jj = result[result['itemtype_jj'] != 'expansion']['objectid_jj'].dropna().loc[result['objectid_jj'] != ''].nunique()
print(distinct_id_count)
print(dist_objectid_ab)
print(dist_objectid_jj)
# Step 3: Calculate the percentage
ab_totalperc = round((dist_objectid_ab / distinct_id_count) * 100, 2)
jj_totalperc = round((dist_objectid_jj / distinct_id_count) * 100, 2)

# Print the result
print(f"Out of the total {distinct_id_count} games listed on Board Games Geek, Adam has {dist_objectid_ab} or {ab_totalperc}% and Jeff has {dist_objectid_jj} or {jj_totalperc}%") 

# Output the 'result' dataframe to an Excel file
#result.to_excel('result.xlsx', index=False)
#ab_jj_outer.to_excel('ab_jj_outer.xlsx', index=False)

20327
1757
298
Out of the total 20327 games listed on Board Games Geek, Adam has 1757 or 8.64% and Jeff has 298 or 1.47%
